In [ ]:
import random
import os
import twitter
import xarray as xr
import numpy as np
import xgcm 
from matplotlib import pyplot as plt
from matplotlib import image
import cmocean
import cartopy.crs as ccrs
import datetime
year = datetime.datetime.now().year
%matplotlib inline

In [ ]:
# overwrite this with the value from the tweet if you want to reproduce an exact plot
seed = abs(hash(datetime.datetime.now()))
print(seed)
random.seed(datetime.datetime.now())

In [ ]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")
list(cat)

In [ ]:
grid_full = cat.LLC4320_grid.to_dask()
grid_full

In [ ]:
nside = 1080
nt = random.randint(0, grid_full.dims['time'])
nface = random.randint(0, grid_full.dims['face'])

frac_ocean = 0.
frac_ocean_thresh = 0.3
while frac_ocean < frac_ocean_thresh:
    i_offset = random.randint(0, grid_full.dims['i'] // nside)
    j_offset = random.randint(0, grid_full.dims['j'] // nside)
    i_slice = slice(nside * i_offset, nside * (i_offset + 1)) 
    j_slice = slice(nside * j_offset, nside * (j_offset + 1))
    selector = dict(time=nt, face=nface, i=i_slice, j=j_slice,
                    i_g=i_slice, j_g=j_slice)
    grid_ds = grid_full.isel(**selector)
    frac_ocean = grid_ds.hFacC.mean().values.item()
    if np.isnan(frac_ocean):
        frac_ocean = 0.
frac_ocean

In [ ]:
grid_ds.Depth.plot()

In [ ]:
ssh = cat.LLC4320_SSH(chunks=False).to_dask().isel(time=nt, face=nface, i=i_slice, j=j_slice)
sst = cat.LLC4320_SST(chunks=False).to_dask().isel(time=nt, face=nface, i=i_slice, j=j_slice)
# sss has some extra metadata
sss = cat.LLC4320_SSS(chunks=False).to_dask().isel(time=nt, face=nface, i=i_slice, j=j_slice)[['SSS']].reset_coords(drop=True)
u = cat.LLC4320_SSU(chunks=False).to_dask().isel(time=nt, face=nface, i_g=i_slice, j=j_slice)
v = cat.LLC4320_SSV(chunks=False).to_dask().isel(time=nt, face=nface, i=i_slice, j_g=j_slice)
ds = xr.merge([grid_ds, ssh, sst, sss, u, v])
# vertical coordiantes are not helpful
ds = ds.drop(['Z', 'Zl', 'Zp1', 'Zu', 'k', 'k_l', 'k_p1', 'PHrefF', 'drC'])
ds

In [ ]:
# fix wraparound lons

def maybe_wrap_lon(lon):
    if abs(lon.min() - lon.max()) < 180:
        return lon
    else:
        return lon.where(lon > 0, other=(lon + 360))
    
ds.coords['XC'] = maybe_wrap_lon(ds.XC)
ds.coords['YG'] = maybe_wrap_lon(ds.YG)

In [ ]:
grid = xgcm.Grid(ds, periodic=False)
grid

In [ ]:
ds['eke'] = 0.5 * (grid.interp(ds.U**2, 'X', boundary='extend')
             + grid.interp(ds.V**2, 'Y', boundary='extend'))

ds['zeta'] = 1e4 * (-grid.diff(ds.U * ds.dxC, 'Y', boundary='extend') +
                    grid.diff(ds.V * ds.dyC, 'X', boundary='extend'))/ds.rAz

ds['div'] = (grid.diff(ds.U * ds.dxC, 'X', boundary='extend') +
             grid.diff(ds.V * ds.dyC, 'Y', boundary='extend'))/ds.rA

In [ ]:
# fix some metadata
ds.zeta.attrs['units'] = r'$10^{-4}$ s$^{-1}$'
ds.zeta.attrs['long_name'] = 'Vorticity'

ds.SST.attrs['units'] = r'$^\circ$C'
ds.SST.attrs['long_name'] = 'Sea Surface Temperature'

ds.SSS.attrs['units'] = r'PSU'
ds.SSS.attrs['long_name'] = 'Sea Surface Salinity'

ds.Eta.attrs['units'] = r'm'
ds.Eta.attrs['long_name'] = 'Sea Surface Height'

In [ ]:
central_lon = ds.XC.mean().values.item()
central_lat = ds.YC.mean().values.item()

lon_min = ds.XC.min().values.item()
lon_max = ds.XC.max().values.item()
lon_range = lon_max - lon_min

lat_min = ds.YC.min().values.item()
lat_max = ds.YC.max().values.item()
lat_range = lat_max - lat_min

proj = ccrs.Orthographic(central_longitude=central_lon,
                         central_latitude=central_lat)

date_str = np.datetime_as_string(ds.time.values, timezone='UTC', unit='m')

location = f'{central_lon:3.1f}, {central_lat:3.1f} | {date_str}'
print(location)

In [ ]:
# tiles near equator are square
# towards pole, both dimensions contract
scale_lon = 2 + 0.75 * abs(np.deg2rad(central_lat))
scale_lat = 2 + 0.3 * abs(np.deg2rad(central_lat))

print(scale_lon, scale_lat)

extent = [central_lon - lon_range/scale_lon, central_lon + lon_range/scale_lon,
          central_lat - lat_range/scale_lat, central_lat + lat_range/scale_lat]

In [ ]:
plt.rcParams['font.size'] = 16

img_url = "https://raw.githubusercontent.com/pangeo-data/branding/master/logo/v_small_pangeo_logo.png"
logo_img = image.imread(img_url)

def plot(da, clip_extent=True, **kwargs):
    xdim = 'XC' if 'i' in da.dims else 'XG'
    ydim = 'YC' if 'j' in da.dims else 'YG'
    
    fig = plt.figure(figsize=(13, 10))
    ax = fig.add_axes([0, 0.02, 1, 0.91],
                      projection=ccrs.Orthographic(central_lon, central_lat))
    ax.background_patch.set_facecolor('0.6')
    if clip_extent:
        ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines()
    
    da.plot(ax=ax, x=xdim, y=ydim, transform=ccrs.PlateCarree(), **kwargs)
    ax.set_title(f'LLC4320 {da.long_name} | {location}')
    
    logo_axis = fig.add_axes([0.81, 0.017, 0.12, 0.05])
    fig.text(0.815, 0.1, f'© Ryan Abernathey\n{year} CC BY',
             fontdict={'size': 11})
    logo_axis.imshow(logo_img, interpolation='hanning')
    logo_axis.axis('off')


In [ ]:
plot(ds.SST, center=False, robust=True, cmap=cmocean.cm.thermal,
     cbar_kwargs={'shrink': 0.5})
plt.savefig('SST.png')

In [ ]:
plot(ds.SSS, center=False, robust=True, cmap=cmocean.cm.haline,
     cbar_kwargs={'shrink': 0.4})
plt.savefig('SSS.png')

In [ ]:
plot(ds.Eta, center=False, robust=True, cmap=cmocean.cm.dense_r,
     cbar_kwargs={'shrink': 0.4})
plt.savefig('SSH.png')

In [ ]:
plot(ds.zeta, robust=True, cmap=cmocean.cm.curl, cbar_kwargs={'shrink': 0.4})
plt.savefig('Vort.png')

In [ ]:
consumer_key, consumer_secret, access_key, access_secret = os.environ['TWITTER_SECRET'].split(':')

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_key,
                  access_token_secret=access_secret)
api

In [ ]:
media = []
for fname in ['SST.png', 'SSS.png', 'SSH.png', 'Vort.png']:
    with open(fname, 'rb') as f:
        media.append(api.UploadMediaChunked(f))

In [ ]:
caption = (f'LLC4320 Ocean Simulation\n'
           f'Random seed: {seed}\n'
           f'Coordinates: {location}\n'
           'https://github.com/rabernat/poseidon-bot\n')
api.PostUpdate(caption, media=media)